# Enem 2020

In [ ]:
# importa as bibliotecas
import zipfile 
import requests
from io import BytesIO
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import count, col, lit, mean
import mysql.connector as msql
from mysql.connector import Error

In [ ]:
# cria um diretório para armazenamento dos dados do ENEM
base_path = "./enem2020"
os.makedirs(base_path, exist_ok=True)

# define a url e faz o download dos dados
url = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"
filebytes = BytesIO(
    requests.get(url, stream=True, timeout=3, verify=False).content
)


In [ ]:
# Extrai o conteúdo do zipfile
myzip = zipfile.ZipFile(filebytes)
myzip.extractall(base_path)

# Leitura e processamento com Pyspark

In [ ]:
# Define o caminho do arquivo
file_path = os.path.join("enem2020", "DADOS", "MICRODADOS_ENEM_2020.csv")
file_path

In [ ]:
# Cria a sessão
spark = SparkSession.builder.appName("ENEM").getOrCreate()

In [ ]:
# Leitura dos dados com Spark
enem = (
    spark
    .read
    .format("csv")
    .options(header=True, delimiter=";", encoding="latin1", inferSchema=True)
    .load(file_path)
)

In [ ]:
# Cria uma view da tabela
enem.createOrReplaceTempView("enem")

# Indicadores

### Qual a escola com a maior média de notas?

In [ ]:
"""
Não foi possível identificar
"""

### Qual o aluno com a maior média de notas e o valor dessa média?

In [ ]:
questao2 = spark.sql("""
    SELECT NU_INSCRICAO as inscricao, 
           (NU_NOTA_CN 
            + NU_NOTA_CH 
            + NU_NOTA_LC 
            + NU_NOTA_MT 
            + NU_NOTA_COMP1 
            + NU_NOTA_COMP2 
            + NU_NOTA_COMP3 
            + NU_NOTA_COMP4 
            + NU_NOTA_COMP5 
            + NU_NOTA_REDACAO)/10
    media_notas
    FROM enem
    ORDER BY 2 DESC
    LIMIT 1
  
""")

In [ ]:
"""
       +------------+-----------+
       |   inscricao|media_notas|
       +------------+-----------+
       |200005996961|     527.29|
       +------------+-----------+
"""
questao2.show()

### Qual a média geral?

In [ ]:
questao3 = spark.sql("""
    SELECT 
    ROUND((AVG(NU_NOTA_CN)
        + AVG(NU_NOTA_CH) 
        + AVG(NU_NOTA_LC) 
        + AVG(NU_NOTA_MT) 
        + AVG(NU_NOTA_COMP1) 
        + AVG(NU_NOTA_COMP2) 
        + AVG(NU_NOTA_COMP3)
        + AVG(NU_NOTA_COMP4)
        + AVG(NU_NOTA_COMP5)
        + AVG(NU_NOTA_REDACAO))/10,2)
    media_geral
    FROM enem
    
""")

In [ ]:
"""
      +-----------+
      |media_geral|
      +-----------+
      |     319.28|
      +-----------+
"""
questao3.show()

### Qual o % de Ausentes?

In [ ]:
questao4 = spark.sql("""
    SELECT ROUND(
        SUM(CASE 
            WHEN TP_PRESENCA_CN = 0 THEN 1
            WHEN TP_PRESENCA_CH = 0 THEN 1
            WHEN TP_PRESENCA_LC = 0 THEN 1
            WHEN TP_PRESENCA_MT = 0 THEN 1
            ELSE 0
        END)*100/COUNT(NU_INSCRICAO),2) as percentual_ausente
    FROM enem 
""")

In [ ]:
"""
      +------------------+
      |percentual_ausente|
      +------------------+
      |             55.21|
      +------------------+
"""
questao4.show()

### Qual o número total de Inscritos?

In [ ]:
questao5 = spark.sql("""
    SELECT COUNT (distinct NU_INSCRICAO) as NUM_inscritos
    FROM enem
""")

In [ ]:
"""
      +-------------+
      |NUM_inscritos|
      +-------------+
      |      5783109|
      +-------------+
"""
questao5.show()

### Qual a média por disciplina?

In [ ]:
questao6 = spark.sql("""
    SELECT ROUND(AVG(NU_NOTA_CN),2) as media_CN,
           ROUND(AVG(NU_NOTA_CH),2) as media_CH,
           ROUND(AVG(NU_NOTA_LC),2) as media_LC,
           ROUND(AVG(NU_NOTA_MT),2) as media_MT
    FROM enem
""")

In [ ]:
"""
      +--------+--------+--------+--------+
      |media_CN|media_CH|media_LC|media_MT|
      +--------+--------+--------+--------+
      |  490.41|  511.15|   523.8|  520.58|
      +--------+--------+--------+--------+
"""
questao6.show()

### Qual a média por Sexo?

In [ ]:
questao7 = spark.sql("""
    SELECT TP_SEXO as sexo,
    ROUND((AVG(NU_NOTA_CN)
        + AVG(NU_NOTA_CH) 
        + AVG(NU_NOTA_LC) 
        + AVG(NU_NOTA_MT) 
        + AVG(NU_NOTA_COMP1) 
        + AVG(NU_NOTA_COMP2) 
        + AVG(NU_NOTA_COMP3)
        + AVG(NU_NOTA_COMP4)
        + AVG(NU_NOTA_COMP5)
        + AVG(NU_NOTA_REDACAO))/10,2)
    media_geral
    FROM enem
    WHERE TP_SEXO IN ('M', 'F')
    GROUP BY TP_SEXO
""")

In [ ]:
"""
      +----+-----------+
      |sexo|media_geral|
      +----+-----------+
      |   F|      318.2|
      |   M|     320.92|
      +----+-----------+
"""
questao7.show()

### Qual a média por Etnia?

In [ ]:
questao8 = spark.sql("""
     SELECT TP_COR_RACA as Etnia,
    ROUND((AVG(NU_NOTA_CN)
        + AVG(NU_NOTA_CH) 
        + AVG(NU_NOTA_LC) 
        + AVG(NU_NOTA_MT) 
        + AVG(NU_NOTA_COMP1) 
        + AVG(NU_NOTA_COMP2) 
        + AVG(NU_NOTA_COMP3)
        + AVG(NU_NOTA_COMP4)
        + AVG(NU_NOTA_COMP5)
        + AVG(NU_NOTA_REDACAO))/10,2)
    media_geral
    FROM enem
    WHERE TP_COR_RACA IN ('0', '1', '2', '3', '4', '5')
    GROUP BY TP_COR_RACA
    ORDER BY media_geral DESC
""") 

In [ ]:
"""
      +-----+-----------+
      |Etnia|media_geral|
      +-----+-----------+
      |    1|     338.45|
      |    0|     322.48|
      |    4|      318.2|
      |    3|     308.04|
      |    2|     302.64|
      |    5|     284.18|
      +-----+-----------+
"""
questao8.show()

# Exportação para o MySQL

In [ ]:
# Conecta ao MySql e cria a database mesha_db
try:
    conn = msql.connect(host='localhost', user='root',  
                        password='password')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE mesha_db")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
# Importa os módulos
spark = SparkSession.builder.config("spark.jars.packages","mysql:mysql-connector-java:5.1.47") \
    .master("local").appName("PySpark_MySQL_test2").getOrCreate()

In [ ]:
# Cria a tabela local_aplicacao_d e salva o dataframe nela
enem.select("NU_INSCRICAO","CO_MUNICIPIO_PROVA","NO_MUNICIPIO_PROVA","SG_UF_PROVA").write.format("jdbc").option("url", "jdbc:mysql://127.0.0.1:3306/mesha_db") \
	.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "local_aplicacao_d") \
	.option("user", "root").option("password", "password").save()

In [ ]:
# Cria a tabela participante_d e salva o dataframe nela
enem.select("NU_INSCRICAO","NU_ANO","TP_FAIXA_ETARIA","TP_SEXO","TP_ESTADO_CIVIL","TP_COR_RACA","TP_NACIONALIDADE","TP_ST_CONCLUSAO","TP_ANO_CONCLUIU","TP_ESCOLA","TP_ENSINO","IN_TREINEIRO").write.format("jdbc").option("url", "jdbc:mysql://127.0.0.1:3306/mesha_db") \
	.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "participante_d") \
	.option("user", "root").option("password", "password").save()

In [ ]:
# Cria a tabela questionario_d e salva o dataframe nela
enem.select("NU_INSCRICAO","Q001","Q002","Q003","Q004","Q005","Q006","Q007","Q008","Q009","Q010","Q011","Q012","Q013","Q014","Q015","Q016","Q017","Q018","Q019","Q020","Q021","Q022","Q023","Q024","Q025").write.format("jdbc").option("url", "jdbc:mysql://127.0.0.1:3306/mesha_db") \
	.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "questionario_d") \
	.option("user", "root").option("password", "password").save()

In [ ]:
# Cria a tabela escola_d e salva o dataframe nela
enem.select("NU_INSCRICAO","CO_MUNICIPIO_ESC","NO_MUNICIPIO_ESC","CO_UF_ESC","SG_UF_ESC","TP_DEPENDENCIA_ADM_ESC","TP_LOCALIZACAO_ESC","TP_SIT_FUNC_ESC").write.format("jdbc").option("url", "jdbc:mysql://127.0.0.1:3306/mesha_db") \
	.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "escola_d") \
	.option("user", "root").option("password", "password").save()

In [ ]:
# Cria a tabela prova_objetiva_F e salva o dataframe nela
enem.select("NU_INSCRICAO","TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT","CO_PROVA_CN","CO_PROVA_CH","CO_PROVA_LC","CO_PROVA_MT","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC","NU_NOTA_MT","TP_LINGUA","TP_STATUS_REDACAO","NU_NOTA_COMP1","NU_NOTA_COMP2","NU_NOTA_COMP3","NU_NOTA_COMP4","NU_NOTA_COMP5","NU_NOTA_REDACAO").write.format("jdbc").option("url", "jdbc:mysql://127.0.0.1:3306/mesha_db") \
	.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "prova_objetiva_f") \
	.option("user", "root").option("password", "password").save()